## Simple Gen AI  APP with langchain and openai

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

In [2]:
## Data Ingestion-- From the website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [4]:
docs=loader.load()

In [10]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset tra

In [11]:
### Load Data---> Docs --> Divide our text into chunks-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [12]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformatio

In [13]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents=documents,embedding=embeddings)

In [15]:
vectorstoredb

In [19]:
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Lets start by setting limits on our production usage, since that is where the majority of spend comes from.\nSetting a good total traces limit\u200b\nPicking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your assumptions before setting a limit.\nFor example:\n\nCurrent Load: Our gen AI application is called between 1.2-1.5 times per second, and each API request has a trace associated with it,\nmeaning we log around 100,000-130,000 traces per day\nExpected Growth in Load: We expect to double in size in the near future.\n\nFrom these assumptions, we can do a quick back-of-the-envelope calculation to get a good limit of:\nlimit = current_load_per_day * expected_growth * days/month      = 130,000 * 2 * 30      = 7,800,000 traces / month\nWe click on the edit icon on the right side of the table for our Prod row, and can enter this limit as follows:'

In [20]:
from langchain_openai import ChatOpenAI 
llm=ChatOpenAI(model='gpt-4o')

In [21]:
## Retrieval Chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """ 
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' \nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x118206f90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1190f9cd0>, root_client=<openai.OpenAI object at 0x118200410>, root_async_client=<openai.AsyncOpenAI object at 0x1182036b0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents

In [36]:
from langchain_core.documents import Document
document_chain.invoke({"input":query,"context":[Document(page_content="Change project level retention defaults")]})

'To change project-level retention defaults, you would typically go into the settings or configuration section of your project management or version control tool. Within the project settings, you should find an option to adjust the retention policies or defaults. However, specific steps can vary depending on the tool you are using. Look for options related to data retention, storage management, or project lifecycle management to make the necessary adjustments.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given questin.

In [24]:
### Input-->Retriever-->vectorstoredb
vectorstoredb

In [25]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [26]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11811fce0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' \nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | ChatOpenAI

In [33]:
### Get the response from the LLM
response=retriever_chain.invoke({"input":"Change project level retention defaults"})

In [34]:
response['answer']

"Based on the provided context, here is the information regarding optimizing data retention and managing costs using LangSmith's system:\n\n1. **Change Project-Level Retention Defaults**: You can navigate to individual project pages to modify data retention settings if the existing projects have not changed their data retention settings.\n\n2. **Setting Retention at the Organization Level**: Changing the organization-level retention defaults for new projects will affect all new projects in all workspaces. This change is aimed at optimizing retention settings to cut down costs and ensure that excessive data retention does not lead to unexpected charges.\n\n3. **Cost Management with Retention Policies**: LangSmith charges differently for traces based on their data retention duration. Short-lived traces are significantly less expensive than those retained for extended periods. By optimizing data retention policies, you can potentially reduce expenses from ~40k per month to ~7.5k per month

In [35]:
response

{'input': 'Change project level retention defaults',
 'context': [Document(id='cd30d997-fdec-4edb-a0fb-6dd16b9f8d68', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Change project level retention defaults\u200b\nOur existing projects have not changed their data retention settings, so we can change these on the individual project pages.\nWe navigate to Projects -> <your project name>, click the data retention drop down, and modify it to base retention. As\nwith the organization level setting, this will only affect retention (and pricing) for traces going forward.'),
  Document(id='52d18a10-7ac3-4588-a4b2-099c116b466c', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangS